In [11]:
import pandas as pd
import time
import re
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# 🟡 파일 경로 & 출력 폴더 경로
input_file = "kakao_data_1.csv"  # 너가 주는 파일명으로 바꿔줘
output_dir = "output_chunks"
os.makedirs(output_dir, exist_ok=True)

# 🟡 100개 단위 크롤링
df = pd.read_csv(input_file)
title_ids = df['title_id'].tolist()[:30]  # 앞의 30개만 테스트
chunk_size = 100

# 🟡 Selenium 옵션
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

for i in range(0, len(title_ids), chunk_size):
    chunk_ids = title_ids[i:i+chunk_size]
    records = []

    for title_id in chunk_ids:
        url = f"https://page.kakao.com/content/{title_id}"
        driver.get(url)
        time.sleep(2)  # 충분한 로딩 대기

        try:
            # 현재 작품 제목
            title_elem = driver.find_element(By.CSS_SELECTOR, "span.font-large3-bold")
            current_title = title_elem.text.strip()
            current_title_clean = re.sub(r'(\[.*?\]|\(.*?\))', '', current_title).strip()

            # 작가의 다른 작품 목록
            try:
                header_div = driver.find_element(By.XPATH, "//div[text()='이 작가의 다른 작품']")
                ancestor_div = header_div.find_element(By.XPATH, "./ancestor::div[contains(@class, 'w-full overflow-hidden')]")
                parent_of_list = ancestor_div.find_element(By.XPATH, "./following-sibling::div//ul")
                li_list = parent_of_list.find_elements(By.TAG_NAME, "li")
            except Exception:
                li_list = []  # 작품이 아예 없는 경우

            normalized_title_set = set()

            for li in li_list:
                try:
                    info_div = li.find_element(By.CSS_SELECTOR, "div[aria-hidden='true'].w-full.space-y-4pxr.pb-4pxr.pr-8pxr.pt-8pxr.h-76pxr")
                    media_elem = info_div.find_element(By.CSS_SELECTOR, "span.break-all.align-middle")
                    media = media_elem.text.strip()
                    if media == "웹소설":
                        continue

                    title_elem = info_div.find_element(By.CSS_SELECTOR, "div.font-small1.line-clamp-2.break-all.text-el-60")
                    raw_title = title_elem.text.strip()
                    clean_title = re.sub(r'(\[.*?\]|\(.*?\))', '', raw_title).strip()
                    normalized_title_set.add(clean_title)
                except Exception:
                    continue

            # 현재 작품도 추가
            normalized_title_set.add(current_title_clean)

            # ✅ 작품 수 기록
            records.append({
                "title_id": title_id,
                "작품개수": len(normalized_title_set)
            })

            print(f"[{title_id}] 크롤링 완료 - 작품수: {len(normalized_title_set)}")

        except Exception as e:
            print(f"[{title_id}] Error: {e}")
            records.append({
                "title_id": title_id,
                "작품개수": 0  # 실패했으면 0으로 기록
            })

    # ✅ 100개씩 저장
    chunk_num = (i // chunk_size) + 1
    output_file = os.path.join(output_dir, f"작가_작품수_{chunk_num:04d}.csv")
    pd.DataFrame(records).to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"✅ {output_file} 저장 완료")

driver.quit()
print("🎉 모든 크롤링 끝!")


WebDriverException: Message: Service /Users/gana/.wdm/drivers/chromedriver/mac64/136.0.7103.113/chromedriver-mac-arm64/chromedriver unexpectedly exited. Status code was: -9


In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument("--headless")

service = Service(ChromeDriverManager().install())
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

driver = webdriver.Remote(
    command_executor="http://localhost:4444/wd/hub",
    desired_capabilities=DesiredCapabilities.CHROME
)

driver.get("https://www.google.com")
print(driver.title)
driver.quit()

TypeError: WebDriver.__init__() got an unexpected keyword argument 'desired_capabilities'